In [1]:
%run GpuOptions.ipynb
%run BuildTrainingData.ipynb
%run TrendTools.ipynb

Found 6086 images belonging to 4 classes.
Found 1521 images belonging to 4 classes.


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.layers import *
import common.model as m

classes = training_data_generator.class_indices
data, _ = training_data_generator.next()
shape = data[0].shape
inputs = tf.keras.Input(shape=shape)

outputs = m.chained(
    # Entry Flow
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    m.expand(
        flow=lambda previous_activation, size: tf.keras.layers.add([
            m.chained(
                m.duplicate(
                    layers=lambda: [
                        tf.keras.layers.Activation('relu'),
                        tf.keras.layers.SeparableConv2D(filters=size, kernel_size=3, padding='same'),
                        tf.keras.layers.BatchNormalization(),
                    ],
                    count=2
                ),
                tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),
            )(previous_activation),
            tf.keras.layers.Conv2D(filters=size, kernel_size=1, strides=2, padding='same')(previous_activation),
        ]),
        values=[128, 256, 728],
    ),

    # # Middle Flow
    m.expand(
        flow=lambda previous_activation, _: tf.keras.layers.add([
            m.duplicate(
                layers=lambda: [
                    tf.keras.layers.Activation('relu'),
                    tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                ],
                count=3,
            )(previous_activation),
            previous_activation,
        ]),
        values=[0] * 8
    ),

    # # Exit Flow
    lambda previous_activation: tf.keras.layers.add([
        m.chained(
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.SeparableConv2D(filters=1024, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),
        )(previous_activation),
        tf.keras.layers.Conv2D(filters=1024, kernel_size=1, strides=2, padding='same')(previous_activation),
    ]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.SeparableConv2D(filters=1024, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(classes), activation='linear'),
    tf.keras.layers.Dropout(0.2),
)(inputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.build(input_shape=(None, *shape))
model.compile(
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.008),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'],
)
if os.path.exists('isthemountainout.best.h5'):
    model.load_weights('isthemountainout.best.h5')
elif os.path.exists('isthemountainout.h5'):
    model.load_weights('isthemountainout.h5')
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [3]:
from datetime import datetime

model.fit(
    training_data_generator,
    epochs=700,
    verbose=True,
    steps_per_epoch=training_data_generator.samples // 
        training_data_generator.batch_size,
    validation_data=validation_data_generator,
    validation_steps=validation_data_generator.samples // 
        validation_data_generator.batch_size,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            'isthemountainout.best.h5',
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            verbose=True),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=100,
            restore_best_weights=True,
            verbose=True),
        # tf.keras.callbacks.CSVLogger(os.path.join('logs', 'isthemountainout.training.csv')),
        tf.keras.callbacks.TensorBoard(
            log_dir=os.path.join('logs', 'fit', datetime.now().strftime('%Y%m%d%H%M%S')),
            update_freq=50,
            write_images=True,
            write_graph=True,
            embeddings_freq=10),
        m.LogConfusionMatrixCallback(
            model=model,
            datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)\
                .flow_from_directory(data_directory, batch_size=3096, shuffle=True, target_size=image_size),
            logdir=os.path.join('logs', 'image'))
    ])

Found 7607 images belonging to 4 classes.
Epoch 1/700
   1/1521 [..............................] - ETA: 0s - loss: 1.0066 - accuracy: 0.5000WARNING:tensorflow:From C:\Users\Taylor\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1521/1521 [==============================] - ETA: 0s - loss: 0.6240 - accuracy: 0.8588
Epoch 00001: val_loss improved from inf to 0.56974, saving model to isthemountainout.best.h5
1521/1521 [==============================] - 179s 118ms/step - loss: 0.6240 - accuracy: 0.8588 - val_loss: 0.5697 - val_accuracy: 0.8947
Epoch 2/700
1521/1521 [==============================] - ETA: 0s - loss: 0.6018 - accuracy: 0.8678
Epoch 00002: val_loss did not improve from 0.56974
1521/1521 [==============================] - 187s 123ms/step - loss: 0.6018 - accuracy: 0.8678 - v

Epoch 27/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5454 - accuracy: 0.8931
Epoch 00027: val_loss did not improve from 0.56974
1521/1521 [==============================] - 174s 114ms/step - loss: 0.5454 - accuracy: 0.8931 - val_loss: 0.6870 - val_accuracy: 0.8625
Epoch 28/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5344 - accuracy: 0.8964
Epoch 00028: val_loss did not improve from 0.56974
1521/1521 [==============================] - 184s 121ms/step - loss: 0.5344 - accuracy: 0.8964 - val_loss: 0.7128 - val_accuracy: 0.8605
Epoch 29/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5376 - accuracy: 0.8963
Epoch 00029: val_loss did not improve from 0.56974
1521/1521 [==============================] - 184s 121ms/step - loss: 0.5376 - accuracy: 0.8963 - val_loss: 0.7028 - val_accuracy: 0.8684
Epoch 30/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5279 - accuracy: 0.9051
Epoch 00030: val_loss did not improv

1521/1521 [==============================] - 174s 114ms/step - loss: 0.5262 - accuracy: 0.9051 - val_loss: 0.6875 - val_accuracy: 0.8671
Epoch 56/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5265 - accuracy: 0.9022
Epoch 00056: val_loss did not improve from 0.56974
1521/1521 [==============================] - 174s 114ms/step - loss: 0.5265 - accuracy: 0.9022 - val_loss: 0.6756 - val_accuracy: 0.8684
Epoch 57/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5148 - accuracy: 0.9107
Epoch 00057: val_loss did not improve from 0.56974
1521/1521 [==============================] - 165s 108ms/step - loss: 0.5148 - accuracy: 0.9107 - val_loss: 0.6582 - val_accuracy: 0.8783
Epoch 58/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5151 - accuracy: 0.9129
Epoch 00058: val_loss did not improve from 0.56974
1521/1521 [==============================] - 173s 114ms/step - loss: 0.5151 - accuracy: 0.9129 - val_loss: 0.7058 - val_accuracy: 0.8612

Epoch 84/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5221 - accuracy: 0.9074
Epoch 00084: val_loss did not improve from 0.56974
1521/1521 [==============================] - 160s 105ms/step - loss: 0.5221 - accuracy: 0.9074 - val_loss: 0.7657 - val_accuracy: 0.8546
Epoch 85/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5207 - accuracy: 0.9047
Epoch 00085: val_loss did not improve from 0.56974
1521/1521 [==============================] - 168s 111ms/step - loss: 0.5207 - accuracy: 0.9047 - val_loss: 0.6865 - val_accuracy: 0.8763
Epoch 86/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5235 - accuracy: 0.9018
Epoch 00086: val_loss did not improve from 0.56974
1521/1521 [==============================] - 169s 111ms/step - loss: 0.5235 - accuracy: 0.9018 - val_loss: 0.7262 - val_accuracy: 0.8651
Epoch 87/700
1521/1521 [==============================] - ETA: 0s - loss: 0.5195 - accuracy: 0.9013
Epoch 00087: val_loss did not improv